In [2]:
#load library
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso,LassoCV,LassoLarsCV,LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score,KFold,RepeatedKFold,GridSearchCV
from scipy.stats import pearsonr, ttest_ind, levene
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn import neighbors
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
from scipy.stats import pearsonr, ttest_ind, levene
import itertools
import time
from array import array
from sklearn.model_selection import RepeatedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as shc
from openpyxl import Workbook
import xlwings as xw
from openpyxl.styles import Font
import openpyxl

# read total file and save to train set and test set


In [ ]:

# read file
df = pd.read_excel(feature all.xlsx)
#Extract label=0 from the training set
df = df.loc[(df['group'] == 'train') & (df['label'] == 0)]  
df = df.drop(['group', 'label'], axis=1)
df.to_excel(train_0.xlsx', index=False)



# read file
df = pd.read_excel(feature all.xlsx)
#Extract label=1 from the training set
df = df.loc[(df['group'] == 'train') & (df['label'] == 1)] 
df = df.drop(['group', 'label'], axis=1)

df.to_excel(train_1.xlsx', index=False)




# read file
df = pd.read_excel(feature all.xlsx)
#Extract label=0 from the test set
df = df.loc[(df['group'] == 'test') & (df['label'] == 0)]  
df = df.drop(['group', 'label'], axis=1)
df.to_excel(test_0.xlsx', index=False)



# read file
df = pd.read_excel(feature all.xlsx)
#Extract label=1 from the test set
df = df.loc[(df['group'] == 'test') & (df['label'] == 1)] #Delete TMV, preserve PTV
df = df.drop(['group', 'label'], axis=1)

df.to_excel(test_1.xlsx', index=False)




# create training set

In [ ]:
xlsx1_filePath = 'train_0.xlsx'
xlsx2_filePath = 'train_1.xlsx'
data_1 = pd.read_excel(xlsx1_filePath)
data_2 = pd.read_excel(xlsx2_filePath)
rows_1,__ = data_1.shape
rows_2,__ = data_2.shape
data_1.insert(0,'label',[0]*rows_1)
data_2.insert(0,'label',[1]*rows_2)
data = pd.concat([data_1,data_2])
data = shuffle(data)
data = data.fillna(0)
X= data[data.columns[1:]]
y= data['label']
colNames = X.columns
X= X.astype(np.float64)
X= StandardScaler().fit_transform(X)
X= pd.DataFrame(X)
X.columns = colNames
print(data)

# t-test

In [ ]:
#t-test for feature selection
index = []
for colName in data.columns[1:]:
    if levene(data_1[colName],data_2[colName])[1] > 0.05:
        if ttest_ind(data_1[colName],data_2[colName])[1] < 0.05:
            index.append(colName)
    else:
        if ttest_ind(data_1[colName],data_2[colName],equal_var = False)[1] < 0.05:
            index.append(colName)
print(len(index))

# standardization

In [ ]:
# to select the 'positive' features
if 'label' not in index:index = ['label']+index
data_1 = data_1[index]
data_2 = data_2[index]
data = pd.concat([data_1,data_2])
data = shuffle(data)
data.index = range(len(data))#re-label after mixure
X = data[data.columns[1:]]
y = data['label']
X = X.apply(pd.to_numeric,errors = 'ignore') # transform the type of the data 
colNames = X.columns # to read the feature's name
X = X.fillna(0)
X = X.astype(np.float64)
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.columns = colNames
X_raw = X 


# lasso

In [ ]:
# lasso for further feature selection
alphas = np.logspace(-3,0,50)
model_lassoCV = LassoCV(alphas = alphas, cv =5, max_iter = 100000).fit(X,y)

print(model_lassoCV.alpha_)
coef = pd.Series(model_lassoCV.coef_,index = X.columns)
print('Lasso picked ' + str(sum(coef !=0))+' variables and eliminated the other ' + str(sum(coef == 0)))

index = coef[coef != 0].index


X = X[index]
X.head()
print(coef[coef !=0])


#Delete the rest of the data in the train set and keep only the filtered features
df1 = pd.read_excel(train_0.xlsx)
df2 = pd.read_excel(train_1.xlsx)

df1 = df1.filter(index)
df2 = df2.filter(index)

df1.to_excel(train_0.xlsx, index=False)
df2.to_excel(train_1.xlsx, index=False)


#Delete the rest of the data in the test set and keep only the filtered features
df1 = pd.read_excel(test_0.xlsx)
df2 = pd.read_excel(test_1.xlsx)

df1 = df1.filter(index)
df2 = df2.filter(index)

df1.to_excel(test_0.xlsx, index=False)
df2.to_excel(test_1.xlsx, index=False)

# plot



In [ ]:
# Plot
plt.figure(figsize=(5, 5), dpi= 80)  
plt.title("Radiomic Dendograms", fontsize=22)  
dend = shc.dendrogram(shc.linkage(X[:].T, method='ward'), labels=X.columns, color_threshold=10)  #参数调整
plt.xticks(fontsize=12,rotation = 60, ha = 'right')
plt.show()

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

MSEs = model_lassoCV.mse_path_
'''
MSEs_mean, MSE_std = [],[]
for i in range(len(MESs)):
    MSEs_mean.append(MSEs[i].mean())
    MSEs_std.append(MSEs[i].std())
'''

MSEs_mean = np.apply_along_axis(np.mean,1,MSEs)
MSEs_std = np.apply_along_axis(np.std,1,MSEs)

plt.figure() # dpi = 300
plt.rcParams['savefig.dpi'] = 1200 
plt.rcParams['figure.dpi'] = 1200 
plt.errorbar(model_lassoCV.alphas_,MSEs_mean
            ,yerr = MSEs_std
            ,fmt = 'o' 
            ,ms = 3 # dot size
            ,mfc = 'r' # dot color
            ,mec = 'r' # dot margin color
            ,ecolor = 'lightblue' 
            ,elinewidth = 2 # error bar width
            ,capsize = 4  # cap length of error bar 
            ,capthick = 1) 
plt.semilogx()
plt.axvline(model_lassoCV.alpha_,color = 'black',ls = '--')
plt.xlabel('Lamda')
plt.ylabel('MSE')
ax = plt.gca()
y_major_locator = ticker.MultipleLocator(0.1)
ax.yaxis.set_major_locator(y_major_locator)
plt.savefig("lasso1.jpg")
plt.show()

coefs = model_lassoCV.path(X_raw,y,alphas = alphas, max_iter = 10000000)[1].T
plt.figure()
plt.semilogx(model_lassoCV.alphas_,coefs,'-')
plt.axvline(model_lassoCV.alpha_,color = 'black',ls = '--')
plt.xlabel('Lamda')
plt.ylabel('Coefficient')
plt.savefig("lasso2.jpg")
plt.show()

# define train and test set

In [ ]:
#X_train，y_train
xlsx1_filePath = 'train_0.xlsx'
xlsx2_filePath = 'train_1.xlsx'
data_1 = pd.read_excel(xlsx1_filePath)
data_2 = pd.read_excel(xlsx2_filePath)
rows_1,__ = data_1.shape
rows_2,__ = data_2.shape
data_1.insert(0,'label',[0]*rows_1)
data_2.insert(0,'label',[1]*rows_2)
data = pd.concat([data_1,data_2])
data = shuffle(data)
data = data.fillna(0)
X_train= data[data.columns[1:]]
y_train= data['label']
colNames = X_train.columns
X_train= X_train.astype(np.float64)
X_train= StandardScaler().fit_transform(X_train)
X_train= pd.DataFrame(X_train)

In [ ]:
#X_test，y_test
xlsx1_filePath = 'test_0.xlsx'
xlsx2_filePath = 'test_1.xlsx'
data_1 = pd.read_excel(xlsx1_filePath)
data_2 = pd.read_excel(xlsx2_filePath)
rows_1,__ = data_1.shape
rows_2,__ = data_2.shape
data_1.insert(0,'label',[0]*rows_1)
data_2.insert(0,'label',[1]*rows_2)
data = pd.concat([data_1,data_2])
data = shuffle(data)
data = data.fillna(0)
X_test = data[data.columns[1:]]
y_test= data['label']
colNames = X_test.columns
X_test= X_test.astype(np.float64)
X_test= StandardScaler().fit_transform(X_test)
X_test= pd.DataFrame(X_test)

X_test.to_csv('X_test.csv', index=None)
y_test.to_csv('y_test.csv', index=None)

# LR model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression

model_LR = LogisticRegression(C=0.1)
model_LR.fit(X_train, y_train)

y_prob_train = model_LR.predict_proba(X_train)[:, 1]
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_prob_train)

y_prob_test = model_LR.predict_proba(X_test)[:, 1]
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_prob_test)

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Test set AUC: {:.2f}".format(grid.score(X_test, y_test)))

model_LR_best = LogisticRegression(C=grid.best_params_['C'])
model_LR_best.fit(X_train, y_train)

y_prob_train_best = model_LR_best.predict_proba(X_train)[:, 1]
fpr_train_best, tpr_train_best, thresholds_train_best = roc_curve(y_train, y_prob_train_best)

y_prob_test_best = model_LR_best.predict_proba(X_test)[:, 1]
fpr_test_best, tpr_test_best, thresholds_test_best = roc_curve(y_test, y_prob_test_best)

plt.plot(fpr_train_best, tpr_train_best)
plt.plot(fpr_test_best, tpr_test_best)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(['Train', 'Test'])
plt.text(0.5, 0.3, f'Train AUC: {roc_auc_score(y_train, y_prob_train_best):.2f}\nTest AUC: {roc_auc_score(y_test, y_prob_test_best):.2f}', fontsize=12)
plt.show()

print('coefficient：', model_LR.coef_)
print('coefficient：', model_LR.将_)

from sklearn.metrics import confusion_matrix
y_pred_test = model_LR.predict(X_test)
cm = confusion_matrix(y_test, y_pred_test)
TP = cm[0][0]
FP = cm[0][1]
TN = cm[1][1]
FN = cm[1][0]
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
NPV = TN / (TN + FN)
PPV = TP / (TP + FP)
accuracy = (TP + TN) / (TP + FP + TN + FN)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("NPV:", NPV)
print("PPV:", PPV)
print("Accuracy:", accuracy)
print(y_prob_test.reshape(-1, 1))

     
                    

import pandas as pd

# Convert y_train to dataframe format
y_test_df = pd.DataFrame(y_test)
# Save y_train_df to specified path
y_test_df.to_excel('y_train.xlsx')

# Get the intercept and coefficients of the logistic regression model
intercept = model_LR.intercept_
coef = model_LR.coef_

# Convert the coefficients to a one-dimensional array
coef = coef.ravel()

# Get the names of the features
features = X.columns

# Build the logistic regression formula string
formula = "y = 1 / (1 + exp(-(" + str(intercept)
for i in range(len(features)):
    formula += " + " + str(coef[i]) + " * " + features[i]
formula += ")))"

# Print the logistic regression formula
print(formula)